## Tasks (Polyglot)

In [ ]:
//// test

open testing

## task_name

In [ ]:
nominal task_name = string

## manual_scheduling

In [ ]:
union manual_scheduling =
    | WithSuggestion
    | WithoutSuggestion

## recurrency_offset

In [ ]:
union recurrency_offset =
    | Days : i32
    | Weeks : i32
    | Months : i32

## day_of_week

In [ ]:
union day_of_week =
    | Sunday
    | Monday
    | Tuesday
    | Wednesday
    | Thursday
    | Friday
    | Saturday

## month

In [ ]:
union month =
    | January
    | February
    | March
    | April
    | May
    | June
    | July
    | August
    | September
    | October
    | November
    | December

## day

In [ ]:
nominal day = i32

## year

In [ ]:
nominal year = i32

## fixed_recurrency

In [ ]:
union fixed_recurrency =
    | Weekly : day_of_week
    | Monthly : day
    | Yearly : day * month

## recurrency

In [ ]:
union recurrency =
    | Offset : recurrency_offset
    | Fixed : list fixed_recurrency

## scheduling

In [ ]:
union scheduling =
    | Manual : manual_scheduling
    | Recurrent : recurrency

## task

In [ ]:
type task =
    {
        name : task_name
        scheduling : scheduling
    }

## date

In [ ]:
type date =
    {
        year : year
        month : month
        day : day
    }

## status

In [ ]:
union status =
    | Postponed : option ()

## event

In [ ]:
type event =
    {
        date : date
        status : status
    }

## task_template

In [ ]:
type task_template =
    {
        task : task
        events : list event
    }

## get_tasks (test)

In [ ]:
//// test

inl get_tasks () : list task_template =
    [
        {
            task =
                {
                    name = task_name "01"
                    scheduling = Manual WithSuggestion
                }
            events = []
        }
        {
            task =
                {
                    name = task_name "02"
                    scheduling = Manual WithSuggestion
                }
            events = []
        }
        {
            task =
                {
                    name = task_name "03"
                    scheduling = Manual WithSuggestion
                }
            events = []
        }
    ]

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

get_tasks ()
|> sm'.format_debug
|> _assert sm'.contains "01"

.py output (Cuda):
{ name = __assert; actual = 01; expected = UH2_1(v0='01', v1=US1_0(v0=US0_0()), v2=UH1_0(), v3=UH2_1(v0='02', v1=US1_0(v0=US0_0()), v2=UH1_0(), v3=UH2_1(v0='03', v1=US1_0(v0=US0_0()), v2=UH1_0(), v3=UH2_0()))) }

.rs output:
{ name = __assert; actual = 01; expected = UH2_1("01", US1_0(US0_0), UH1_0, UH2_1("02", US1_0(US0_0), UH1_0, UH2_1("03", US1_0(US0_0), UH1_0, UH2_0))) }

.ts output:
{ name = __assert; actual = 01; expected = UH2_1 (01, US1_0 US0_0, UH1_0, UH2_1 (02, US1_0 US0_0, UH1_0, UH2_1 (03, US1_0 US0_0, UH1_0, UH2_0))) }

.py output:
{ name = __assert; actual = 01; expected = UH2_1 ("01", US1_0 US0_0, UH1_0, UH2_1 ("02", US1_0 US0_0, UH1_0, UH2_1 ("03", US1_0 US0_0, UH1_0, UH2_0))) }



.fsx output:
{ name = __assert; actual = 01; expected = UH2_1
  ("01", US1_0 US0_0, UH1_0,
   UH2_1 ("02", US1_0 US0_0, UH1_0, UH2_1 ("03", US1_0 US0_0, UH1_0, UH2_0))) }


In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

get_tasks ()
|> listm'.try_item 0i32
|> fun (Some task) => task.task.name
|> _assert_eq (task_name "01")

.py output (Cuda):
{ name = __assert_eq; actual = 01; expected = 01 }

.rs output:
{ name = __assert_eq; actual = "01"; expected = "01" }

.ts output:
{ name = __assert_eq; actual = 01; expected = 01 }

.py output:
{ name = __assert_eq; actual = 01; expected = 01 }



.fsx output:
{ name = __assert_eq; actual = "01"; expected = "01" }


In [ ]:
//// test
///! fsharp
////! cuda
////! typescript
////! python
///// print_code

inl print padding cols =
    ({ lines = []; last_lines = []; max_acc = 0i32 }, cols)
    ||> listm.fold fun { last_lines max_acc } lines =>
        inl { count max } =
            (lines, { count = 0i32; max = 0i32 })
            ||> listm.foldBack fun line { count max } => {
                count = count + 1
                max =
                    inl len = line |> sm'.length
                    if len > max
                    then len
                    else max
            }
        inl { lines } =
            (lines, { lines = []; i = 0i32 })
            ||> listm.foldBack fun line { lines i } => {
                lines =
                    inl last_line =
                        last_lines
                        |> listm'.try_item (count - i - 1)
                        |> optionm'.default_with fun () =>
                            " " |> sm'.replicate max_acc
                    inl line =
                        if padding = 0
                        then line
                        else
                            inl padding = " " |> sm'.replicate padding
                            $'$"{!line}{!padding}"'
                    inl line = line |> sm'.pad_right (max + padding) ' '
                    $'$"{!last_line}{!line}"' :: lines
                i = i + 1
            }
        {
            lines
            last_lines = lines
            max_acc = max_acc + max + padding
        }
    |> fun x => x.lines
    |> listm'.box
    |> seq.of_list'
    |> sm'.concat "\n"

inl col () =
    [ "Task" ]
    ++ (
        get_tasks ()
        |> listm.map fun task =>
            inl (task_name name) = task.task.name
            name
    )

inl cols () =
    [
        col ()
        col ()
        [ "a"; "b"; "c"; "d"; "e" ]
    ]

inl main () =
    cols ()
    |> print 1i32
    |> console.write_line

Task Task a 
01   01   b 
02   02   c 
03   03   d 
          e 
